In [1]:
import urllib
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader

from PIL import Image
from time import gmtime, strftime
from torch.utils.tensorboard import SummaryWriter

In [2]:
model = torch.hub.load('pytorch/vision:v0.12.0', 'googlenet', pretrained=True)

Using cache found in C:\Users\Taynet Gaming/.cache\torch\hub\pytorch_vision_v0.12.0
C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Miniconda3\envs\ml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
eurosat_dir = './EuroSAT/2750'

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

folder = ImageFolder(eurosat_dir, transform=preprocess)


train, validate, test = random_split(folder, [16200, 5400, 5400], generator=torch.Generator().manual_seed(42))
print('Training set length: ', len(train))
print('Validation set length: ', len(validate))
print('Test set length: ', len(test))
torch.cuda.is_available()

Training set length:  16200
Validation set length:  5400
Test set length:  5400


True

In [4]:
num_classes = 10 # EuroSAT has 10 classes

url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
input_image = Image.open(filename)


input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

nodel = model.cuda()
input_batch.cuda()

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# set model parameters fixed
set_parameter_requires_grad(model, True)
model.fc = nn.Linear(1024, num_classes)
model.train()


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [5]:
BATCH_SIZE = 128

# training data loaders
train_loader = DataLoader(
    train, batch_size=BATCH_SIZE, shuffle=True
)
# validation data loaders
valid_loader = DataLoader(
    validate, batch_size=BATCH_SIZE, shuffle=False
)

In [6]:
# training
def train(model, trainloader, optimizer, criterion):
    model = model.cuda()
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in enumerate(trainloader):
        counter += 1
        image, labels = data
        image = image.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs, 1)
        x = (preds == labels).sum().item()
        train_running_correct += (preds == labels).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc

# validate
def validate(model, valid_loader, criterion):
    model = model.cuda()
    model.eval()
    print('Validating')
    val_running_loss = 0.0
    val_running_correct = 0
    counter = 0
    for i, data in enumerate(valid_loader):
        counter += 1
        image, labels = data
        image = image.cuda()
        labels = labels.cuda()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        val_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs, 1)
        x = (preds == labels).sum().item()
        val_running_correct += (preds == labels).sum().item()

    # loss and accuracy for the complete epoch
    epoch_loss = val_running_loss / counter
    epoch_acc = 100. * (val_running_correct / len(valid_loader.dataset))
    return epoch_loss, epoch_acc

def save_model(epochs, model, optimizer):
    torch.save(model.state_dict(), f'./model_epoch_{epochs}.pth')
    torch.save(optimizer.state_dict(), f'./opt_epoch_{epochs}.pth')



In [7]:
writer = SummaryWriter()

epochs = 15
optimizer = optim.Adam(model.parameters(), lr=1e-2)

loss = torch.nn.CrossEntropyLoss()

# lists to keep track of losses and accuracies
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
# start the training
for epoch in range(epochs):
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()), f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(model, train_loader, 
                                              optimizer, loss)
    valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader,  
                                                   loss)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    writer.add_scalar("Loss/train", train_epoch_loss, epoch)
    writer.add_scalar("Accuracy/train", train_epoch_acc, epoch)
    
    writer.add_scalar("Loss/validate", valid_epoch_loss, epoch)
    writer.add_scalar("Accuracy/validate", valid_epoch_acc, epoch)
    
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)
    #time.sleep(5)
    save_model(epoch, model, optimizer)
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()), f"[INFO]: Model saved")
    
# save the trained model weights

# save the loss and accuracy plots
#save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('TRAINING COMPLETE')
writer.flush()
writer.close()

2022-07-01 11:55:14 [INFO]: Epoch 1 of 15
Training
Validating
Training loss: 0.604, training acc: 80.093
Validation loss: 0.308, validation acc: 89.315
--------------------------------------------------
2022-07-01 11:56:08 [INFO]: Model saved
2022-07-01 11:56:08 [INFO]: Epoch 2 of 15
Training
Validating
Training loss: 0.329, training acc: 88.710
Validation loss: 0.270, validation acc: 90.667
--------------------------------------------------
2022-07-01 11:56:55 [INFO]: Model saved
2022-07-01 11:56:55 [INFO]: Epoch 3 of 15
Training
Validating
Training loss: 0.320, training acc: 88.833
Validation loss: 0.279, validation acc: 90.370
--------------------------------------------------
2022-07-01 11:57:43 [INFO]: Model saved
2022-07-01 11:57:43 [INFO]: Epoch 4 of 15
Training
Validating
Training loss: 0.327, training acc: 88.969
Validation loss: 0.250, validation acc: 91.648
--------------------------------------------------
2022-07-01 11:58:30 [INFO]: Model saved
2022-07-01 11:58:30 [INFO]: 

In [8]:
# test on the test set
test_loader = DataLoader(
    test, batch_size=BATCH_SIZE, shuffle=False
)
test_loss, test_acc = validate(model, test_loader,  loss)
print(f"Test loss: {test_loss:.3f}, Test acc: {test_acc:.3f}")


Validating
Test loss: 0.292, Test acc: 90.870


In [9]:
# tensorboard https://tensorboard.dev/experiment/LggebNtpS9uNcbbSwnLXNA/